
# CP322 — Assignment 2: Fake vs Real News Classification

**Dataset:** Fake and Real News — Kaggle  
**Goal:** Build classifiers (Naive Bayes, Logistic Regression, MLP) to detect fake news from text.  
**Student:** Jay Patel


In [3]:
import sys
print(sys.executable)

/usr/local/bin/python3.11


In [4]:
pip install nltk


Note: you may need to restart the kernel to use updated packages.


In [1]:
import nltk
print(nltk.__version__)

3.9.2


In [5]:
pip install torch torchvision torchaudio

  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 MB 57.8 MB/s  0:00:01m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 33.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 808.4/808.4 kB 40.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 70.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 87.8 MB/s  0:00:00
Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)
  Attempting uninstall: typing-extensions━━━━━━━ 0/9 [mpmath]
    Found existing installation: typing_extensions 4.9.09 [mpmath]
    Uninstalling typing_extensions-4.9.0:━━━ 0/9 [mpmath]
      Successfully uninstalled typing_extensions-4.9.0━━━━━━━━━━━━ 1/9 [typing-extensions]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9/9 [torchaudio]9 [torchaudio]]s]
Note: you may need to restart the kernel to use updated packages.


In [1]:
import torch
print(torch.__version__)

2.9.0


## 0) Setup & Imports

In [2]:


import os, re, math, random, string, time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                             f1_score, roc_auc_score, roc_curve, confusion_matrix)

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader





[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1006)>
[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1006)>
[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1006)>


## 1) Data Exploration & Preprocessing (5 pts)


### 1.1 Load & Merge
Load `True.csv` (label=0) and `Fake.csv` (label=1). Keep `title` and `text` only.


In [6]:

TRUE_PATH = "True.csv"  # TODO: set
FAKE_PATH = "Fake.csv"  # TODO: set

true_df = pd.read_csv(TRUE_PATH)[['title','text']].assign(label=0)
fake_df = pd.read_csv(FAKE_PATH)[['title','text']].assign(label=1)
df = pd.concat([true_df, fake_df], ignore_index=True).sample(frac=1.0, random_state=42).reset_index(drop=True)
print(df.shape); df.head()


(44898, 3)


,title,text,label
0,BREAKING: GOP Chairman Grassley Has Had Enoug...,"Donald Trump s White House is in chaos, and th...",1
1,Failed GOP Candidates Remembered In Hilarious...,Now that Donald Trump is the presumptive GOP n...,1
2,Mike Pence’s New DC Neighbors Are HILARIOUSLY...,Mike Pence is a huge homophobe. He supports ex...,1
3,California AG pledges to defend birth control ...,SAN FRANCISCO (Reuters) - California Attorney ...,0
4,AZ RANCHERS Living On US-Mexico Border Destroy...,Twisted reasoning is all that comes from Pelos...,1


### 1.2 Class Distribution & Missing Values

In [ ]:

counts = df['label'].value_counts().sort_index()
print("Counts:", counts.to_dict())

fig, ax = plt.subplots()
ax.bar(['Real (0)','Fake (1)'], [counts.get(0,0), counts.get(1,0)])
ax.set_title("Class Distribution"); ax.set_ylabel("Count")
plt.show()

print("\nMissing values:"); print(df.isna().sum())
before = len(df)
df = df.dropna(subset=['title','text'])
print(f"Dropped {before - len(df)} rows with nulls in title/text.")


### 1.3 Text Preprocessing (tokenize, lowercase, stopwords, lemmatize)

In [ ]:

lemm = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def clean_text(s):
    if not isinstance(s, str): return ""
    s = s.lower()
    tokens = word_tokenize(s)
    kept = []
    for t in tokens:
        if t.isalpha() and t not in stop_words:
            kept.append(lemm.lemmatize(t))
    return " ".join(kept)

df['text_clean'] = (df['title'].fillna('') + " " + df['text'].fillna('')).apply(clean_text)
df[['title','text','text_clean','label']].head()


## 2) Feature Engineering (1 pt)

In [ ]:

tfidf = TfidfVectorizer(max_features=5000, ngram_range=(1,1))
X = tfidf.fit_transform(df['text_clean'])
y = df['label'].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Train:", X_train.shape, " Test:", X_test.shape)


## 3) Naive Bayes & Logistic Regression (2 pts)

In [ ]:

def metrics_report(y_true, y_prob, thresh=0.5, name="Model"):
    y_pred = (y_prob >= thresh).astype(int)
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, zero_division=0)
    rec = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    try:
        auc = roc_auc_score(y_true, y_prob)
    except ValueError:
        auc = float('nan')
    print(f"{name}: Acc={acc:.4f} Prec={prec:.4f} Rec={rec:.4f} F1={f1:.4f} AUC={auc:.4f}")
    return acc, prec, rec, f1, auc

# GaussianNB (requires dense)
nb = GaussianNB().fit(X_train.toarray(), y_train)
nb_prob = nb.predict_proba(X_test.toarray())[:,1]
_ = metrics_report(y_test, nb_prob, name="GaussianNB")

# Logistic Regression
logreg = LogisticRegression(max_iter=100)
logreg.fit(X_train, y_train)
lr_prob = logreg.predict_proba(X_test)[:,1]
_ = metrics_report(y_test, lr_prob, name="LogisticRegression")


## 4) Multilayer Perceptron (PyTorch) (2 pts)

In [ ]:

Xtr = torch.from_numpy(X_train.toarray()).float()
ytr = torch.from_numpy(y_train.reshape(-1,1)).float()
Xte = torch.from_numpy(X_test.toarray()).float()
yte = torch.from_numpy(y_test.reshape(-1,1)).float()

train_loader = DataLoader(TensorDataset(Xtr, ytr), batch_size=128, shuffle=True)
test_loader  = DataLoader(TensorDataset(Xte, yte), batch_size=256, shuffle=False)

class MLP(nn.Module):
    def __init__(self, in_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, 128), nn.ReLU(),
            nn.Linear(128, 128), nn.ReLU(),
            nn.Linear(128, 1), nn.Sigmoid()
        )
    def forward(self, x): return self.net(x)

model = MLP(X_train.shape[1])
crit = nn.BCELoss()
opt = torch.optim.Adam(model.parameters(), lr=1e-3)

def eval_probs(m, loader):
    m.eval(); ps=[]; ys=[]
    with torch.no_grad():
        for xb,yb in loader:
            pb = m(xb); ps.append(pb.numpy()); ys.append(yb.numpy())
    return np.vstack(ps), np.vstack(ys)

EPOCHS = 20
for ep in range(1, EPOCHS+1):
    model.train(); running=0.0
    for xb,yb in train_loader:
        opt.zero_grad(); out = model(xb); loss = crit(out, yb)
        loss.backward(); opt.step(); running += loss.item()*xb.size(0)
    probs, ys = eval_probs(model, test_loader)
    try:
        auc = roc_auc_score(ys, probs)
    except ValueError:
        auc = float('nan')
    print(f"Epoch {ep:02d} | loss={running/len(Xtr):.4f} | test AUC={auc:.4f}")

mlp_prob, _ys = eval_probs(model, test_loader)
_ = metrics_report(y_test, mlp_prob, name="MLP")


## 5) Evaluation (5 pts)

In [ ]:

def plot_confusion(y_true, y_prob, title="Confusion Matrix", threshold=0.5):
    y_pred = (y_prob >= threshold).astype(int)
    cm = confusion_matrix(y_true, y_pred)
    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest')
    ax.set_title(title); ax.set_xlabel("Predicted"); ax.set_ylabel("Actual")
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, str(cm[i, j]), ha='center', va='center')
    plt.tight_layout(); plt.show()

def plot_roc_multi(y_true, probs_list, labels):
    fig, ax = plt.subplots()
    for probs, lbl in zip(probs_list, labels):
        fpr, tpr, _ = roc_curve(y_true, probs)
        ax.plot(fpr, tpr, label=lbl)
    ax.plot([0,1],[0,1],'--')
    ax.set_title("ROC Curves"); ax.set_xlabel("False Positive Rate"); ax.set_ylabel("True Positive Rate"); ax.legend()
    plt.tight_layout(); plt.show()

plot_confusion(y_test, nb_prob,  "GaussianNB — Confusion Matrix")
plot_confusion(y_test, lr_prob,  "Logistic Regression — Confusion Matrix")
plot_confusion(y_test, mlp_prob, "MLP — Confusion Matrix")

plot_roc_multi(y_test, [nb_prob, lr_prob, mlp_prob], ["GaussianNB","LogReg","MLP"])


## 6) Discussion


- Summarize metrics (Accuracy, Precision, Recall, F1, AUC) for each model.
- Discuss false positives vs. false negatives and their implications.
- Suggest improvements (bigram features, class weights, threshold tuning, transformer models).
